In [1]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [2]:
# Generating Empty Aggregator to be loaded 

setting = 'FedEM'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "celeba"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = '/home/ubuntu/FedEM_Shubh/FedEM/weights/celeba_benign/fedem/'
args_.validation = False
args_.tune_steps = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 229.79it/s]


===> Initializing clients..


100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:31<00:00,  2.56it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.922 | Train Acc: 1.464% |Test Loss: 2.923 | Test Acc: 1.297% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [3]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

In [4]:
# Import Model Weights
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
#     args_.save_path ='weights/final/femnist/fig1_take3/local_adv/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
#     args_.save_path = 'weights/final/femnist/figperturb/fedem_avg_p0_1/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedem_benign/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
    weights = np.load(args_.save_path+"train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [5]:
# Here we will make a dictionary that will hold results
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]

In [6]:
# Perform transfer attack from one client to another and record stats

# Run Measurements for both targetted and untargeted analysis
new_num_models = len(models_test)
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 4.5


for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    
    dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    
    # Perform Attacks
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = 8, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    
    
    
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Untargeted attack
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 10
	 Adv idx: 11
	 Adv idx: 12
	 Adv idx: 13
	 Adv idx: 14
	 Adv idx: 15
	 Adv idx: 16
	 Adv idx: 17
	 Adv idx: 18
	 Adv idx: 19
	 Adv idx: 20
	 Adv idx: 21
	 Adv idx: 22
	 Adv idx: 23
	 Adv idx: 24
	 Adv idx: 25
	 Adv idx: 26
	 Adv idx: 27
	 Adv idx: 28
	 Adv idx: 29
	 Adv idx: 30
	 Adv idx: 31
	 Adv idx: 32
	 Adv idx: 33
	 Adv idx: 34
	 Adv idx: 35
	 Adv idx: 36
	 Adv idx: 37
	 Adv idx: 38
	 Adv idx: 39


In [7]:
# Aggregate Results Across clients 
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [8]:
print(adv_target[:10, :10])

[[1.00 0.62 0.38 0.38 1.00 0.62 0.38 0.62 0.38 1.00]
 [0.98 0.88 0.82 0.75 0.96 0.90 0.57 0.88 0.78 0.98]
 [0.92 0.75 0.82 0.81 0.96 0.92 0.67 0.71 0.85 0.93]
 [0.94 0.75 0.81 0.83 0.94 0.93 0.77 0.74 0.88 0.94]
 [0.94 0.61 0.56 0.44 1.00 0.89 0.39 0.56 0.72 1.00]
 [0.94 0.82 0.82 0.82 1.00 0.94 0.65 0.76 0.88 0.94]
 [0.93 0.46 0.63 0.69 0.91 0.87 0.74 0.44 0.85 0.96]
 [1.00 0.85 0.85 0.80 0.95 0.90 0.70 0.80 0.85 0.95]
 [0.83 0.67 0.72 0.78 0.94 0.94 0.83 0.67 0.94 0.94]
 [1.00 0.33 0.28 0.22 0.94 0.72 0.17 0.33 0.50 1.00]]


In [9]:
def avg_nondiag(array2d):
    d1 = array2d.shape[0]
    d2 = array2d.shape[1]
    
    counter = 0
    val = 0
    
    for i1 in range(d1):
        for i2 in range(d2):
            if i1 != i2:
                if not np.isnan(array2d[i1,i2]):
                    counter+=1
                    val += array2d[i1,i2]
    
    return val/counter

In [10]:
np.isnan(adv_target).sum()

40

In [11]:
adv_target

array([[1.00, 0.62, 0.38, ..., 0.38, 1.00, 1.00],
       [0.98, 0.88, 0.82, ..., 0.53, 0.92, 0.98],
       [0.92, 0.75, 0.82, ..., 0.65, 0.93, 0.95],
       ...,
       [0.91, 0.51, 0.64, ..., 0.81, 0.90, 0.91],
       [0.96, 0.55, 0.58, ..., 0.49, 0.94, 0.99],
       [1.00, 0.50, 0.50, ..., 0.50, 1.00, 1.00]])

In [12]:
print('adv_target:', avg_nondiag(adv_target))
print('adv_miss:', avg_nondiag(adv_miss))
print('orig_acc:', np.mean(np.diagonal(orig_acc)))

adv_target: 0.6757596512469001
adv_miss: 0.1294986568734599
orig_acc: 0.8055869042873383


In [9]:
# Check Results
print(orig_acc[:10,:10])

[[0.45 0.80 0.81 0.77 0.36 0.64 0.80 0.79 0.58 0.42]
 [0.62 0.78 0.77 0.76 0.58 0.68 0.74 0.76 0.64 0.60]
 [0.35 0.83 0.83 0.77 0.24 0.63 0.80 0.81 0.51 0.32]
 [0.38 0.72 0.72 0.70 0.35 0.57 0.70 0.71 0.52 0.39]
 [0.62 0.68 0.68 0.67 0.60 0.61 0.65 0.68 0.62 0.60]
 [0.40 0.78 0.78 0.78 0.36 0.55 0.80 0.77 0.50 0.41]
 [0.18 0.85 0.85 0.82 0.09 0.54 0.82 0.84 0.38 0.17]
 [0.43 0.78 0.78 0.77 0.31 0.60 0.79 0.78 0.52 0.40]
 [0.59 0.75 0.75 0.71 0.50 0.63 0.72 0.74 0.59 0.56]
 [0.42 0.80 0.80 0.79 0.39 0.59 0.80 0.82 0.55 0.42]]


In [12]:
print(adv_acc[:10, :10])

[[0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.41 0.41 0.41 0.41 0.41 0.43 0.41 0.45 0.47 0.41]
 [0.53 0.55 0.54 0.57 0.54 0.63 0.54 0.54 0.68 0.53]
 [0.35 0.38 0.38 0.38 0.38 0.41 0.38 0.41 0.42 0.35]
 [0.17 0.22 0.22 0.22 0.17 0.22 0.22 0.22 0.22 0.17]
 [0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06]
 [0.46 0.48 0.48 0.48 0.48 0.48 0.48 0.48 0.50 0.46]
 [0.40 0.45 0.40 0.45 0.40 0.45 0.40 0.45 0.55 0.40]
 [0.22 0.22 0.17 0.33 0.17 0.33 0.22 0.17 0.33 0.22]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.06 0.00]]


In [13]:
print(adv_miss[:10, :10])

[[0.07 0.10 0.07 0.07 0.07 0.02 0.07 0.07 0.02 0.07]
 [0.12 0.14 0.12 0.14 0.12 0.18 0.12 0.10 0.22 0.12]
 [0.15 0.17 0.14 0.17 0.15 0.18 0.16 0.17 0.22 0.15]
 [0.14 0.12 0.12 0.12 0.13 0.12 0.10 0.09 0.13 0.13]
 [0.11 0.11 0.13 0.13 0.11 0.11 0.11 0.13 0.09 0.11]
 [0.10 0.14 0.10 0.10 0.10 0.00 0.10 0.10 0.00 0.10]
 [0.07 0.07 0.06 0.07 0.07 0.10 0.06 0.06 0.14 0.07]
 [0.15 0.10 0.15 0.10 0.15 0.10 0.10 0.10 0.10 0.15]
 [0.11 0.11 0.11 0.11 0.11 0.06 0.11 0.11 0.06 0.11]
 [0.07 0.07 0.08 0.07 0.06 0.06 0.07 0.08 0.06 0.07]]


In [12]:
method = 'adv'

In [13]:
setting

'FedEM'

In [14]:
np.save(f'results_celeba/{method}/{setting}_orig_acc_transfers', orig_acc)
np.save(f'results_celeba/{method}/{setting}_orig_similarities', orig_sim)
np.save(f'results_celeba/{method}/{setting}_adv_acc_transfers', adv_acc)
np.save(f'results_celeba/{method}/{setting}_adv_similarities_target', adv_sim_target)
np.save(f'results_celeba/{method}/{setting}_adv_similarities_untarget', adv_sim_untarget)
np.save(f'results_celeba/{method}/{setting}_adv_target', adv_target)
np.save(f'results_celeba/{method}/{setting}_adv_miss', adv_miss)

In [12]:
np.save(f'results_celeba_prop/no_prop_orig_acc_transfers', orig_acc)
np.save(f'results_celeba_prop/no_prop_orig_similarities', orig_sim)
np.save(f'results_celeba_prop/no_prop_adv_acc_transfers', adv_acc)
np.save(f'results_celeba_prop/no_prop_adv_similarities_target', adv_sim_target)
np.save(f'results_celeba_prop/no_prop_adv_similarities_untarget', adv_sim_untarget)
np.save(f'results_celeba_prop/no_prop_adv_target', adv_target)
np.save(f'results_celeba_prop/no_prop_adv_miss', adv_miss)

In [13]:
print('orig_acc:', np.mean(np.diagonal(orig_acc)))
print('adv_miss:', avg_nondiag(adv_miss))
print('adv_target:', avg_nondiag(adv_target))

orig_acc: 0.5413639895617962
adv_miss: 0.046813174346700695
adv_target: 4.528985591605306e-05
